**Pre cleaning of microbiome data** 
Preparing 16S data for further analysis such as beta diversity and statistical tests. 
1. add variabel types to meta data
2. filter features/samples that will not be part of any analysis

In [ ]:
library(phyloseq)
library(stringr)
library(ggplot2)
suppressMessages(library(microbiome))
source("/mnt/work/flatberg/projects/GCF-2019-658/analysis/src/microbiome/rules/analysis/notebooks/src/microfiltR/microfiltR_source_code.R")

**Load QIIME2 based taxonomy from phyloseq object (rds file)**

In [ ]:
rds <- readRDS(snakemake@input[[1]])
rds

In [ ]:
#rds.f <- subset_samples(rds, Sample_Biosource == "saft")
df <- sample_data(rds)
ctrl <- rep(FALSE, 70)
ctrl[62:70] <- TRUE
df[ctrl,"Sample_Biosource"] <- "control"
sample_data(rds) <- df
rds.f <- subset_samples(rds, Sample_Biosource == "saft")
rds.f

**Taxonomy requirements**

In [ ]:
rds.f <- subset_taxa(rds,
    Kingdom == "Bacteria" &
    Family  != "mitochondria" &
    Class   != "Chloroplast" &
    Phylum != "Cyanobacteria/Chloroplast"
  )
rds.f = prune_taxa(taxa_sums(rds.f) > 0, rds.f)
rds.f


**Independent feature filtes**  
Functions for paramter tuning independent filtering is from Bryan Brown's microfiltR https://github.com/itsmisterbrown/microfiltR

In [ ]:
as.threshold <- estimate.ASthreshold(ps=rds.f, WST=NULL, minLIB=500, Prange = c(0.025:0.15, 0.025), CVrange = c(1:10, 0.5), RArange = c(5e-6:0.9e-3, 1e-5))

In [ ]:
plot.threshold(as.threshold)

In [ ]:
e3.f <- microfilter(ps=rds.f, WST=NULL, PFT=0.05, RAT=2.5e-5, minLIB=500, return.all = TRUE)
e3.f$filtered.phyloseq

In [ ]:
rds.f <- e3.f$filtered.phyloseq

**Save filtered SILVA**

In [ ]:
saveRDS(rds.f, snakemake@output[[1]])

--------

--------------
